# Load data into an Azure Table

This notebook creates embeddings from the sample data and loads the text and embeddings into an Azure Table. The dataset is a collection of wikipedia articles about the 2020 Summer Olympics that comes from the [OpenAI Cookbook repo](https://github.com/openai/openai-cookbook).

In [1]:
from azure.data.tables import TableServiceClient
import numpy as np
from azure.core.credentials import AzureNamedKeyCredential
import json
import pandas as pd
import os
import time
import openai

# See the notebook below for details on how the data was gathered
# https://github.com/openai/openai-cookbook/blob/main/examples/fine-tuned_qa/olympics-1-collect-data.ipynb
data_path = 'https://cdn.openai.com/API/examples/data/olympics_sections_text.csv'

## 1.0 Setup

To run through this notebook, you'll need to have an [Azure OpenAI Service](https://learn.microsoft.com/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal) resource and an [Azure Table Storage](https://learn.microsoft.com/azure/storage/common/storage-account-create?tabs=azure-portal) resource. 

### 1.1 Make sure the following environmant variables are set

In [2]:
#os.environ['OPENAI_API_KEY'] = '<your Azure OpenAI API key>'
#os.environ['OPENAI_RESOURCE_ENDPOINT'] = '<your Azure OpenAI resource endpoint>'

#os.environ['AZURE_TABLE_KEY'] = '<your Azure Table Storage key>'
#os.environ['AZURE_TABLE_ACCOUNT_NAME'] = '<your Azure Table Storage account name>'

### 1.2 Update setting for your Azure OpenAI Service resource

In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
OPENAI_RESOURCE_ENDPOINT = os.getenv("OPENAI_RESOURCE_ENDPOINT") 

openai.api_type = "azure"
openai.api_key = OPENAI_API_KEY
openai.api_base = OPENAI_RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"

TEXT_SEARCH_EMBEDDING_ENGINE = 'text-search-ada-doc-001'

### 1.3 Create a client for Azure Table Storage

In [4]:
# load key from environement variable
AZURE_TABLE_KEY = os.getenv("AZURE_TABLE_KEY")
TABLE_ACCOUNT_NAME = os.getenv("AZURE_TABLE_ACCOUNT_NAME")
TABLE_NAME = "embeddings"

credential = AzureNamedKeyCredential(TABLE_ACCOUNT_NAME, AZURE_TABLE_KEY)
table_service_client = TableServiceClient(endpoint="https://{}.table.core.windows.net/".format(TABLE_ACCOUNT_NAME), credential=credential)

## 2.0 Load data into a dataframe

In [5]:
documents_df = pd.read_csv(data_path)

print(f"Number of documents: {len(documents_df)}")
documents_df.head()

Number of documents: 3964


,title,heading,content,tokens
0,2020 Summer Olympics,Summary,The 2020 Summer Olympics (Japanese: 2020年夏季オリン...,726
1,2020 Summer Olympics,Host city selection,The International Olympic Committee (IOC) vote...,126
2,2020 Summer Olympics,Impact of the COVID-19 pandemic,"In January 2020, concerns were raised about th...",374
3,2020 Summer Olympics,Qualifying event cancellation and postponement,Concerns about the pandemic began to affect qu...,298
4,2020 Summer Olympics,Effect on doping tests,Mandatory doping tests were being severely res...,163


## 3.0 Load embeddings into Azure Table

### 3.1 Create an Azure Table

In [6]:
# delete the table if needed; this operation takes a couple of minutes
# table_service_client.delete_table(table_name=TABLE_NAME)

In [7]:
# create a table
table_service_client.create_table(table_name=TABLE_NAME)

# get the table client
table_client = table_service_client.get_table_client(table_name=TABLE_NAME)

### 3.2 Create embeddings from the text and loading it into an Azure Table

In [8]:
# Defining a helper function to create embeddings from the text and retry when needed
def get_embedding(text: str, model: str) -> list[float]:
    retries = 0
    max_retries = 5

    while retries < max_retries:
        try:
            result = openai.Embedding.create(
                engine=model,
                input=text
            )
            break
        except openai.error.RateLimitError as e:
            print("Rate limit exceeded, retrying in 12 seconds")
            retries += 1
            time.sleep(12)

    return result["data"][0]["embedding"]

In [9]:
for i, row in documents_df.iloc[0:100].iterrows(): # for simplicity, only processing the first 100 documents

    embedding =  get_embedding(row['content'], TEXT_SEARCH_EMBEDDING_ENGINE)

    # add a row to the table
    table_client.create_entity({
            'PartitionKey': row['title'], 
            'RowKey': str(i),
            'embedding': json.dumps(embedding),
            'content': row['content']
        }
    )

    if i % 25 == 0:
        print(f"Processed {i} documents")


Processed 0 documents
Processed 25 documents
Rate limit exceeded, retrying in 12 seconds
Processed 50 documents
Processed 75 documents
